In [10]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib
%pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-ellx5ct6
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-ellx5ct6
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit ecf72c716f6378a9bb2191ed884b5ce9428dcc81
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
from pathlib import Path
import gc
import torch
import os
import torch
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import (
    derive_face_edges_from_faces
)

In [12]:
from accelerate import notebook_launcher
autoencoder = MeshAutoencoder(
    decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,
    dim_codebook = 192,
    dim_area_embed = 16,
    dim_coor_embed = 16,
    dim_normal_embed = 16,
    dim_angle_embed = 8,
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2)

pkg = torch.load("./mesh-transformer-datasets/16k_autoencoder_229M_0.338.pt")
autoencoder.load_state_dict(pkg['model'], strict = False)

dataset = MeshDataset.load("./mesh-transformer-datasets/objverse_250f_490.7M_all_17561_labels_568425_5_min_x5_aug.npz")
dataset2 = MeshDataset.load("./mesh-transformer-datasets/objverse_250f_98.1M_all_17561_labels_113685_5_min_x1_aug.npz")
dataset.data.extend(dataset2.data)
dataset2 = MeshDataset.load("./mesh-transformer-datasets/shapenet_250f_2.2M_84_labels_2156_10_min_x1_aug.npz")
dataset.data.extend(dataset2.data)
dataset2 = MeshDataset.load("./mesh-transformer-datasets/shapenet_250f_21.9M_84_labels_21560_10_min_x10_aug.npz")
dataset.data.extend(dataset2.data)
dataset.sort_dataset_keys()

[MeshDataset] Loaded 568425 entries
[MeshDataset] Created from 568425 entries
[MeshDataset] Loaded 113685 entries
[MeshDataset] Created from 113685 entries
[MeshDataset] Loaded 2156 entries
[MeshDataset] Created from 2156 entries
[MeshDataset] Loaded 21560 entries
[MeshDataset] Created from 21560 entries


In [13]:
# autoencoder.commit_loss_weight = 0.5
# autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
#                                                 batch_size=32,
#                                                 grad_accum_every =2,
#                                                 learning_rate = 1e-4,
#                                                 checkpoint_every_epoch=1)
# _loss1 = autoencoder_trainer.train(14445,  diplay_graph= False)

In [14]:
from torch.utils.data import Dataset
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from meshgpt_pytorch import (
    MeshAutoencoder,
    MeshTransformer
)

transformer = MeshTransformer(
    autoencoder,
    dim = 768,
    coarse_pre_gateloop_depth =2,
    fine_pre_gateloop_depth= 2,
    attn_depth = 12,
    attn_heads = 12,
    cross_attn_num_mem_kv = 4,
    fine_cross_attend_text = True,
    text_cond_with_film = False,
    num_sos_tokens = 1,
    dropout  = 0.0,
    max_seq_len = 1500,
    fine_attn_depth = 2,
    condition_on_text = True,
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge",
    text_condition_cond_drop_prob = 0.0,
).cuda()


def generate_codes(self, autoencoder : MeshAutoencoder, batch_size = 25):
    total_batches = (len(self.data) + batch_size - 1) // batch_size

    for i in tqdm(range(0, len(self.data), batch_size), total=total_batches):
        batch_data = self.data[i:i+batch_size]

        padded_batch_vertices = pad_sequence([item['vertices'] for item in batch_data], batch_first=True, padding_value=autoencoder.pad_id).cuda()
        padded_batch_faces = pad_sequence([item['faces'] for item in batch_data], batch_first=True, padding_value=autoencoder.pad_id).cuda()
        padded_batch_face_edges = pad_sequence([item['face_edges'] for item in batch_data], batch_first=True, padding_value=autoencoder.pad_id).cuda()

        batch_codes = autoencoder.tokenize(
            vertices=padded_batch_vertices,
            faces=padded_batch_faces,
            face_edges=padded_batch_face_edges
        )


        mask = (batch_codes != autoencoder.pad_id).all(dim=-1)
        for item_idx, (item_codes, item_mask) in enumerate(zip(batch_codes, mask)):
            item_codes_masked = item_codes[item_mask]
            item = batch_data[item_idx]
            item['codes'] = item_codes_masked.to("cpu")

    self.sort_dataset_keys()
    print(f"[MeshDataset] Generated codes for {len(self.data)} entries")

generate_codes(dataset, autoencoder, 350)
dataset.embed_texts(transformer, 1)

/home/fire/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
 42%|████▏     | 838/2017 [04:38<06:43,  2.92it/s]

In [ ]:
from accelerate import notebook_launcher
def training_function(num_processes=2):
    batch_size = 16 # 16 batch = 17-19GB VRAM
    grad_accum_every = 64 / batch_size
    rate = 1e-2
    trainer = MeshTransformerTrainer(model=transformer, warmup_steps=10, grad_accum_every=grad_accum_every,
        num_train_steps=100, dataset=dataset, batch_size=batch_size, learning_rate=rate, checkpoint_every_epoch=1)
    trainer.train(503)

args = ()
notebook_launcher(training_function, args, num_processes=1)

# pkg = dict( model = transformer.state_dict(), )
# torch.save(pkg, str("./MeshGPT-transformer_trained.pt"))

Epoch 1/503:   0%|          | 0/44114 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'shape'